In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [ ]:
data = pd.read_csv('../input/water-quality/waterQuality1.csv')

data = data.drop('ammonia', axis = 1)
data = data.drop('flouride', axis = 1)
data = data.drop('selenium', axis = 1)
data = data.drop('uranium', axis = 1)

data = data[data['is_safe'] != '#NUM!']
data['is_safe'] = data['is_safe'].astype(int)

columns = dict(zip(list(range(data.shape[1])), data.columns.values.tolist()))
is_safe = data['is_safe']

data = pd.DataFrame(StandardScaler().fit_transform(data)).rename(columns = columns)
data['is_safe'] = is_safe
data = data.dropna()

data.head(6)

In [ ]:
sns.heatmap(data = data.corr(), linewidths = .5, annot = False, cmap = "YlGnBu")
plt.show()

1. Понижение размерности и визуализация данных

Примените методы снижения размерности sklearn.decomposition.PCA и
sklearn.manifold.TSNE для визуализации данных с которым вы работали в лабораторной №2 (снижая размерность до двух).

In [ ]:
X = data.drop('is_safe', axis = 1)
Y = data['is_safe']

In [ ]:
X_PCA_pl = PCA(n_components = 2).fit_transform(X)
PCA_data_pl = pd.DataFrame(data = X_PCA_pl, columns = ['PC1', 'PC2'])
PCA_data_pl['is_safe'] = Y
PCA_data_pl.head(6)

In [ ]:
X_TSNE_pl = TSNE(n_components = 2).fit_transform(X)
TSNE_data_pl = pd.DataFrame(data = X_TSNE_pl, columns = ['TSNE1', 'TSNE2'])
TSNE_data_pl['is_safe'] = Y
TSNE_data_pl.head(6)

Отобразите полученные результаты

In [ ]:
sns.heatmap(data = PCA_data_pl.corr(), linewidths = .5, annot = True)
plt.show()

In [ ]:
sns.heatmap(data = TSNE_data_pl.corr(), linewidths = .5, annot = True)
plt.show()

2. Кластерный анализ



1) С помощью алгоритма k-means сделайте квантование изображения (удаление
визуально избыточной информации) с глубиной 64, 32, 16 и 8 уровней. Изображение
выбираете произвольно.


In [ ]:
import cv2

from sklearn.cluster import KMeans
from sklearn.utils import shuffle

from PIL import Image

from sklearn.mixture import GaussianMixture

In [ ]:
img = Image.open('../input/no-disaster-1/no damage 1/mountain/00000006.jpg')
uniqueColors = set()

w, h = img.size
for x in range(w):
    for y in range(h):
        pixel = img.getpixel((x, y))
        uniqueColors.add(pixel)

In [ ]:
def recreate_image(codebook, labels, w, h):
    d = codebook.shape[1]
    image = np.zeros((w, h, d))
    label_idx = 0
    for i in range(w):
        for j in range(h):
            image[i][j] = codebook[labels[label_idx]]
            label_idx += 1
    return image

In [ ]:
pic = cv2.imread('../input/no-disaster-1/no damage 1/mountain/00000006.jpg')[...,::-1] / 255.0

w, h, d = pic.shape
_2d_pic = np.reshape(pic, (w * h, d))
_2d_pic_sample = shuffle(_2d_pic)[:5000]

plt.title(f'Original image ({len(uniqueColors)} colors)')
plt.imshow(pic)
plt.show()

for clusters in [8, 16, 32, 64]:
    kmeans = KMeans(n_clusters = clusters).fit(_2d_pic_sample)
    labels = kmeans.predict(_2d_pic)

    plt.title(f'Quantized image ({clusters} colors, K-Means)')
    plt.imshow(recreate_image(kmeans.cluster_centers_, labels, w, h))
    plt.show()

2) Сгенерируйте набор синтетических данных в виде смеси двух гауссиан, результат объедините в одно множество.

In [ ]:
mean_1 = [1, 2]
cov_1 = [[1, 0], [0, 1]]
x_1 = np.random.multivariate_normal(mean_1, cov_1, 100)

mean_2 = [4, 5]
cov_2 = [[2, 0], [0, 2]]
x_2 = np.random.multivariate_normal(mean_2, cov_2, 100)

x = np.concatenate([x_1, x_2])
labels = np.array([0] * 100 + [1] *  100)

plt.scatter(x_1[:, 0], x_1[:, 1], marker = '^', color='coral')
plt.scatter(x_2[:, 0], x_2[:, 1], marker = 'o', color='indigo')
plt.show()

Разделите смесь с помощью EM алгоритма(sklearn.mixture.GaussianMixture), обратите
внимание на параметр covariance_type.

In [ ]:
gm = GaussianMixture(n_components = 2, covariance_type = 'diag').fit(x)

С помощью атрибутов weights_ и covariances_
восстановите их значения (сравните с оригинальными).

In [ ]:
gm.means_

In [ ]:
gm.covariances_

Визуализируйте результат.

In [ ]:
mean_gm = gm.means_[1]
cov_gm = [[gm.covariances_[1][0], 0], [0, gm.covariances_[1][1]]]
x_gm = np.random.multivariate_normal(mean_gm, cov_gm, 100)

plt.scatter(x_1[:, 0], x_1[:, 1], marker = '^', color='coral')
plt.scatter(x_gm[:, 0], x_gm[:, 1], marker = 'o', color='indigo')

3. Обработка текстовых данных


Загрузите набор текстовых данных, снабженных метками классов. Проведите
предобработку данных (удалите стоп-слова, пунктуацию, проведите нормализацию),
постройте визуализацию наиболее частых слов или n-gram в каждом классе (wordcloud),
извлеките признаки (например sklearn.feature_extraction.text .TfidfVectorizer; или
sklearn.decomposition.TruncatedSVD )

In [ ]:
import re

from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
train = pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv',encoding = 'latin1')
test = pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv',encoding = 'latin1')

df = train.append(test, ignore_index = True)

df = df.drop('UserName', axis = 1)
df = df.drop('ScreenName', axis = 1)
df = df.drop('Location', axis = 1)
df = df.drop('TweetAt', axis = 1)

df.head(6)

In [ ]:
def Sentiment_encode(sentiment):
    if sentiment == 'Extremely Positive' or sentiment == 'Positive':
        return 2
    elif sentiment == 'Extremely Negative' or sentiment == 'Negative':
        return 0
    else:
        return 1

def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def decontraction(text):
    text = re.sub(r"won\'t", " will not", text)
    text = re.sub(r"won\'t've", " will not have", text)
    text = re.sub(r"can\'t", " can not", text)
    text = re.sub(r"don\'t", " do not", text)
    
    text = re.sub(r"can\'t've", " can not have", text)
    text = re.sub(r"ma\'am", " madam", text)
    text = re.sub(r"let\'s", " let us", text)
    text = re.sub(r"ain\'t", " am not", text)
    text = re.sub(r"shan\'t", " shall not", text)
    text = re.sub(r"sha\n't", " shall not", text)
    text = re.sub(r"o\'clock", " of the clock", text)
    text = re.sub(r"y\'all", " you all", text)
    
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"n\'t've", " not have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d've", " would have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll've", " will have", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'re", " are", text)
    return text

def seperate_alphanumeric(text):
    words = text
    words = re.findall(r"[^\W\d_]+|\d+", words)
    return " ".join(words)

def cont_rep_char(text):
    tchr = text.group(0) 
    
    if len(tchr) > 1:
        return tchr[0:2] 

def unique_char(rep, text):
    substitute = re.sub(r'(\w)\1+', rep, text)
    return substitute

def char(text):
    substitute = re.sub(r'[^a-zA-Z]',' ',text)
    return substitute

df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : remove_url(x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : remove_emoji(x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : decontraction(x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : seperate_alphanumeric(x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : unique_char(cont_rep_char,x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : char(x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : x.lower())

df['Sentiment'] = df['Sentiment'].apply(lambda x : Sentiment_encode(x))

df.head(6)

In [ ]:
Positive = df[df['Sentiment'] == 2].OriginalTweet
Neutral  = df[df['Sentiment'] == 1].OriginalTweet
Negative = df[df['Sentiment'] == 0].OriginalTweet

In [ ]:
plt.figure(figsize = (20, 20)) 
wordcould = WordCloud(min_font_size = 3, max_words = 300, width = 1600, height = 680).generate(" ".join(Negative))
plt.imshow(wordcould, interpolation = 'bilinear')
plt.grid(False)

In [ ]:
plt.figure(figsize = (20, 20)) 
wordcould = WordCloud(min_font_size = 3, max_words = 300, width = 1600, height = 680).generate(" ".join(Neutral))
plt.imshow(wordcould, interpolation = 'bilinear')
plt.grid(False)

In [ ]:
plt.figure(figsize = (20, 20)) 
wordcould = WordCloud(min_font_size = 3, max_words = 300, width = 1600, height = 680).generate(" ".join(Positive))
plt.imshow(wordcould, interpolation = 'bilinear')
plt.grid(False)

In [ ]:
def features(data):
        vectorizer = TfidfVectorizer(stop_words = 'english')
        tfidf = vectorizer.fit(data)

        idfs = vectorizer.idf_

        lower_thresh = 3.
        upper_thresh = 6.
        not_often = idfs > lower_thresh
        not_rare = idfs < upper_thresh

        mask = not_often * not_rare

        return np.array(vectorizer.get_feature_names())[mask]

Проведите классификацию текстовых данных, сделайте оценку качества

In [ ]:
X = df['OriginalTweet']
Y = df['Sentiment']

X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.2)

vectorizer = TfidfVectorizer(stop_words = 'english')
tfidf = vectorizer.fit(X_train)

X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)

In [ ]:
features(df['OriginalTweet'])

In [ ]:
clf = RandomForestClassifier(n_estimators = 250, criterion = 'gini', max_features = 'sqrt').fit(X_train, Y_train)

predict = clf.predict(X_test)

In [ ]:
print(classification_report(Y_test, predict))

Примените алгоритм LDA к каждому классу, определите несколько тематик (sklearn.decomposition.LatentDirichletAllocation)


In [ ]:
LDA = LatentDirichletAllocation(n_components = 3,
                                max_iter = 30,
                                n_jobs = 6,
                                learning_method = 'batch',
                                verbose = 1)

count_vect = CountVectorizer(stop_words = 'english')

In [ ]:
classes = [Positive, Neutral, Negative]

topic_words = []

for cl in classes:
    LDA.fit(count_vect.fit_transform(cl))
    top_topic_words = LDA.components_[0].argsort()[-10:]
    for i in top_topic_words:
        topic_words.append(count_vect.get_feature_names()[i])

In [ ]:
p = set(topic_words[:10])
ne = set(topic_words[10:20])
n = set(topic_words[20:])

In [ ]:
p_unique = (p.difference(ne)).difference(n)
p_unique

In [ ]:
ne_unique = (ne.difference(p)).difference(n)
ne_unique

In [ ]:
n_unique = (n.difference(p)).difference(ne)
n_unique

In [ ]:
(p.intersection(ne)).intersection(n)